<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
from omegaconf import OmegaConf
import torch
from pathlib import Path
from datetime import datetime
import json, os, sys
import cv2
import numpy as np
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

from stable_diffusion_utils import load_model_from_config, config, generate, clean_cache
from ds_utils.video_utils import imageio_generate_video

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
opt = config()
opt.ckpt = 'D:/models/stable_diffusion/512-base-ema.ckpt'
opt.config = str(Path.home() / 'Documents/python_workspace/stable-diffusion/configs/stable-diffusion/v2-inference.yaml')
model = load_model_from_config(OmegaConf.load(opt.config), opt.ckpt, opt.full_precision).to(device)

batch_idx = 0
sample_idx = 0

In [ ]:
outputs_path = Path.home() / 'Documents/generated_data/stable_diffusion'
batch_name = f'' # name for subfolder and filenames  {datetime.now().strftime("%Y%m%d_%H%M%S")}_
width_height = [800, 800] 
opt.W, opt.H = width_height
opt.scale = 14 # strength of text prompt
opt.n_steps = 80 # number of diffusion steps
opt.sampler = 'ddim' # ["klms","plms", "ddim"]
opt.ddim_eta = 0 # scale of variance from 0.0 to 1.0
opt.seed = -1 # use -1 for random seed or specify number manually
opt.seamless = False
num_batch_images = 5 # how many images you want to generate in this batch

# Init image
opt.init_img = str('C:\\Users\\User\\Downloads\\img.png') # url or path to an image, or path to a folder to pick random images from
opt.strength = 0.65 # from 0.0 to 1.0 how much the init image is used
opt.skip_frames = 2

# Mask
opt.mask_path = None #str('C:\\Users\\User\\Downloads\\mask.mp4')
opt.invert_mask = True

# Prompt
opt.prompt = """"""


opt.outdir = str(outputs_path / batch_name)
opt.process_config()

# save settings
settings = vars(opt)
os.makedirs(opt.outdir, exist_ok=True)
while os.path.isfile(f"{opt.outdir}/{batch_name}_{batch_idx}_settings.txt"):
    batch_idx += 1
with open(f"{opt.outdir}/{batch_name}_{batch_idx}_settings.txt", "w+", encoding="utf-8") as f:
    json.dump(settings, f, ensure_ascii=False, indent=4)
    
sample_idx = 0
video_extensions =  ['.mp4', '.webm']
# process video
if opt.init_img is not None and Path(opt.init_img).suffix in video_extensions:
    if opt.mask_path is not None:
        assert Path(opt.mask_path).suffix in video_extensions, 'Video mask must be a video itself.'
        mask_video = cv2.VideoCapture(opt.mask_path)
    print('Run on video')
    video = cv2.VideoCapture(opt.init_img)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = video.get(cv2.CAP_PROP_FPS)/opt.skip_frames

    frames_out = []
    for frame_index in tqdm(range(frame_count), desc='Running on video', position=0):
        clean_cache()
        ret, frame = video.read()
        if opt.mask_path is not None:
            _, mask_frame = mask_video.read()
        if (frame_index % opt.skip_frames) != 0:
            continue
        if ret:
            opt.init_img = frame
            if opt.mask_path is not None:
                opt.mask_path = mask_frame
            images = generate(opt, model, batch_name, batch_idx, sample_idx)
            sample_idx += 1
            frames_out.append(images[0][0])
    # generate video
    imageio_generate_video(str(f'{opt.outdir}/{batch_name}_{batch_idx}.mp4'), 
                       [np.array(img) for img in frames_out], fps=fps, format="mp4")
    # remove generated frames
    for filename in (outputs_path / batch_name).glob(f'{batch_name}_{batch_idx}_*.png'):
        os.remove(filename) 
# process image
else:
    for i in range(num_batch_images):
        clean_cache()
        _ = generate(opt, model, batch_name, batch_idx, sample_idx)
        opt.seed += 1
        sample_idx += 1


In [ ]:
from PIL import Image
img = Image.open(str('')).convert('RGB')
img.thumbnail([800, 800], Image.ANTIALIAS)  # maintain aspect ratio
img.size